In [1]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.interpolate import Rbf
from sklearn.model_selection import train_test_split

from localization import utils, dataset
from localization.models.rbf_model import Model

In [2]:
utils.make_deterministic(33)

def radial_log_basis_function(model, r):
    return np.log(r + model.epsilon)

Rbf.radial_log_basis_function = radial_log_basis_function
with open('output/filtered_model.bin', 'rb') as inp:
    model: Model = pickle.load(inp)


In [3]:
num_buildings = 3
num_floors_in_each_building = {0: 4,
                               1: 4,
                               2: 5}

sigma = 6.75
num_samples = 2
new_samples = 1000

rng = np.random.default_rng(33)

In [4]:
data = dataset.load_ujiindoor_loc(data_folder='data/filtered')

### Generation based on RBF $\mu$ and $\phi$

In [5]:
generated_data = pd.DataFrame(columns=data.get_full_df().columns)

for _ in tqdm(range(new_samples)):
    b = rng.integers(0, num_buildings)
    f = rng.integers(0, num_floors_in_each_building[b])
    x = rng.uniform(model.x_building[b], model.x_building[b])
    y = rng.uniform(model.y_building[b], model.y_building[b])

    # gen_data = {}
    # for r in model.mu_rbf[b][f].keys():
    #     mu = model.mu_rbf[b][f][r](x, y)
    #     phi = model.phi_rbf[b][f][r](x, y)
    #     phi = np.where(phi > 1, 1, phi)
    #     gen_data[r] = rng.normal(mu, sigma) * rng.binomial(1, phi)

    gen_data = {r: rng.normal(model.mu_rbf[b][f][r](x,y), sigma) * \
                   rng.binomial(1, np.minimum(model.phi_rbf[b][f][r](x,y), 1)) for r in model.mu_rbf[b][f].keys()}

    gen_data["BUILDINGID"] = b
    gen_data["FLOOR"] = f
    gen_data["SPACEID"] = -1
    gen_data["RELATIVEPOSITION"] = -1
    gen_data["USERID"] = -1
    gen_data["PHONEID"] = -1
    gen_data["TIMESTAMP"] = -1
    gen_data["LATITUDE"] = -1
    gen_data["LONGITUDE"] = -1
    gen_data["x"] = x
    gen_data["y"] = y
    generated_data = pd.concat([generated_data, pd.DataFrame(gen_data)], ignore_index=True)

  4%|▍         | 43/1000 [07:06<2:38:17,  9.92s/it]


KeyboardInterrupt: 

In [ ]:
# generated_data = pd.DataFrame(columns=data.get_full_df().columns)

b = rng.integers(0, num_buildings, size=num_samples)
f = np.array([rng.integers(0, num_floors_in_each_building[building]) for building in b])
x = np.array([rng.uniform(model.x_building[building], model.x_building[building]) for building in b])
y = np.array([rng.uniform(model.y_building[building], model.y_building[building]) for building in b])

# gen_data = {}
# for r in model.mu_rbf[b][f].keys():
#     mu = model.mu_rbf[b][f][r](x, y)
#     phi = model.phi_rbf[b][f][r](x, y)
#     phi = np.where(phi > 1, 1, phi)
#     gen_data[r] = rng.normal(mu, sigma) * rng.binomial(1, phi)

gen_data = {r: rng.normal(model.mu_rbf[b][f][r](x,y), sigma) * \
               rng.binomial(1, np.minimum(model.phi_rbf[b][f][r](x,y), 1)) for r in model.mu_rbf[b][f].keys()}

gen_data["BUILDINGID"] = b
gen_data["FLOOR"] = f
# gen_data["SPACEID"] = -np.ones(num_samples)
# gen_data["RELATIVEPOSITION"] = -np.ones(num_samples)
# gen_data["USERID"] = -np.ones(num_samples)
# gen_data["PHONEID"] = -np.ones(num_samples)
# gen_data["TIMESTAMP"] = -np.ones(num_samples)
# gen_data["LATITUDE"] = -np.ones(num_samples)
# gen_data["LONGITUDE"] = -np.ones(num_samples)
gen_data["x"] = x
gen_data["y"] = y
generated_data = pd.DataFrame(gen_data)

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
generated_data = generated_data.fillna(0.0)
generated_data = generated_data.clip(lower=0)
generated_data.sample(10)

In [ ]:
train_df, test_df = train_test_split(generated_data, test_size=0.2)

In [ ]:
train_df.to_csv('data/generated/trainingData.csv', index=False)
test_df.to_csv('data/generated/validationData.csv', index=False)

### Generation based on already calculated points ($\mu_b$ and $\phi_b$)

In [ ]:
generated_data = pd.DataFrame(columns=data.get_full_df().columns)

for building in range(num_buildings):
    for floor in tqdm(range(model.num_floors_in_each_building[building])):
        filtered_dataset = data.get_floor_data(building=building, floor=floor)
        X_train = filtered_dataset.get_full_df()
        routers = model.get_all_routers_in_this_floor(X_train.columns)
        gen_data = {}

        for router in routers:
            if model.checking_non_null_minimum_percentage_of_samples(X_train, router):
                # X_train = model.get_mu_and_phi_estimation(X_train, router)
                phi_building = model.phi_rbf[building][floor][router]
                phi_building = np.where(phi_building > 1, 1, phi_building)
                mu_building = model.mu_rbf[building][floor][router]

                gen_data[router] = np.random.normal(np.tile(mu_building, num_samples), sigma) * \
                                          np.random.binomial(1, np.tile(phi_building, num_samples))

            # print(gen_data[router])
            # print(gen_data[router].shape)

        # first router of third building of first floor has a different shape
        if building == 2 and floor == 0:
            max_length = max(len(arr) for arr in gen_data.values())
            for key in gen_data:
                if len(gen_data[key]) < max_length:
                    gen_data[key] = np.pad(gen_data[key], (0, max_length - len(gen_data[key])), constant_values=0)

        # get the right values
        gen_data = pd.DataFrame(gen_data)
        # print(generated_data.shape, gen_data.shape)
        gen_data.loc[:, "BUILDINGID"] = [building] * gen_data.shape[0]
        gen_data.loc[:, "FLOOR"] = [floor] * gen_data.shape[0]
        gen_data.loc[:, "SPACEID"] = [0] * gen_data.shape[0]
        gen_data.loc[:, "RELATIVEPOSITION"] = [0] * gen_data.shape[0]
        gen_data.loc[:, "USERID"] = [-1] * gen_data.shape[0]
        gen_data.loc[:, "PHONEID"] = [-1] * gen_data.shape[0]
        gen_data.loc[:, "TIMESTAMP"] = [-1] * gen_data.shape[0]
        gen_data.loc[:, "LATITUDE"] = [0] * gen_data.shape[0]
        gen_data.loc[:, "LONGITUDE"] = [0] * gen_data.shape[0]
        gen_data.loc[:, "x"] = np.tile(model.x_building[building], num_samples)
        gen_data.loc[:, "y"] = np.tile(model.y_building[building], num_samples)
        generated_data = pd.concat([generated_data, pd.DataFrame(gen_data)], ignore_index=True)

100%|██████████| 5/5 [00:35<00:00,  7.18s/it]


In [ ]:
generated_data = generated_data.fillna(0.0)
generated_data = generated_data.clip(lower=0)
generated_data.sample(10)

,WAP001,WAP002,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,WAP011,WAP012,...,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP,x,y
20769,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0,1,1,0,0,0,0,0,0.000000,10.356031
41120,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,7.665740,14.533680,...,0,1,2,0,0,0,0,0,0.636560,0.000000
28798,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0,2,1,0,0,0,0,0,0.000000,0.000000
26655,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0,2,1,0,0,0,0,0,0.000000,28.898784
37325,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0,0,2,0,0,0,0,0,0.000000,4.143140
48268,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,3.581022,17.183868,...,0,2,2,0,0,0,0,0,0.636560,34.761567
44526,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,8.516054,32.029452,...,0,1,2,0,0,0,0,0,0.000000,0.000000
38978,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0,0,2,0,0,0,0,0,15.385154,0.000000
16829,0.0,0.0,0.0,7.754054,0.0,12.717207,0.0,0.0,0.000000,0.000000,...,0,0,1,0,0,0,0,0,17.879972,0.000000
50731,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,1.173060,0.000000,...,0,2,2,0,0,0,0,0,0.000000,31.845526


In [ ]:
train_df, test_df = train_test_split(generated_data, test_size=0.2)

In [ ]:
train_df.to_csv('data/generated/trainingData.csv', index=False)
test_df.to_csv('data/generated/validationData.csv', index=False)

: 

In [ ]:
generated_data

### Generation based on probs

In [ ]:
p_xy = 1/model.power_probability_masks[0][0]['WAP007'][0].shape[0]
probs = []
for p in range(106):
    p_p = model.power_prior_probability_distribution[0][0]['WAP007'][p]
    p_xy_given_bfrp = model.power_probability_masks[0][0]['WAP007'][p]
    p_p_given_bfrxy = p_xy_given_bfrp *  p_p / p_xy
    probs.append(p_p_given_bfrxy)

probs = np.array(probs)

In [ ]:
print(probs.shape)
print(probs[:][0].shape)

In [ ]:
np.random.choice(106, 20, p=probs[:,0]/probs[:,0].sum())

In [ ]:
data = dataset.load_ujiindoor_loc(data_folder='data')
data_fil = data.get_floor_data(building=0, floor=0, reset_means=True)

x_building, y_building = model.construct_building_map(data_fil, 0)

In [ ]:
X_train = data_fil.get_full_df()

In [ ]:
X_train.shape